# Financial ML

In [0]:
import numpy as np
import pandas as pd
import sklearn
import datetime

In [2]:
try:
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    link = 'https://drive.google.com/open?id=1Xl9krWLvAUF0IMxFA0hqF2HwG5AejwTN'
    fluff, id = link.split('=')
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('hourAll.csv')  
    data = pd.read_csv('hourAll.csv')
except:
    print('cant colab')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
try:
    from google.colab import files
    uploaded = files.upload()
except:
    print('cant colab')

## Data

In [0]:
class TAQ():
    """TAQ object generated from WRDS database TAQ trade csv file.
    Stores initial CSV as pd dataframe and renames columns"""

    def __init__(self, path='null', data=None):
        if path != 'null':
            self.taqPath = path
            self.rawData = pd.read_csv(path)
        elif data is not None:
            self.rawData = data

        self.data = self.preprocess()
        self.timeBars = self.makeTimeBars()

    def make_timestamp(self, level = 'Min'):
        combined = self.rawData['DATE'].apply(str) + ' ' + self.rawData['TIME_M']
        timestamp = pd.to_datetime(combined)
        return timestamp.dt.floor(level)

    def preprocess(self, level = 'Min'):
        data = self.rawData[['SYM_ROOT', 'EX', 'SIZE', 'PRICE']]
        data = data.rename(columns={"SYM_ROOT": "ticker", "EX": 'exchange', "SIZE" : 'volume', "PRICE": 'price'})
        data['priceXvolume'] = data['price'] * data['volume']
        data['timestamp'] = self.make_timestamp(level)
        data['date'] = data['timestamp'].dt.date
        data['ticks'] = (data['price'].diff()/abs(data['price'].diff())).fillna(method = 'ffill').fillna(1)
        return data

    def makeTimeBars(self):
        """Will create TimeBars using level specified in pre-processing"""

        groupData = self.data.groupby(['ticker', 'timestamp'])
        volsum = groupData['volume'].sum()
        psum = groupData['priceXvolume'].sum()
        vwap = psum / volsum
        o = groupData['price'].first()
        c = groupData['price'].last()
        high = groupData['price'].max()
        low = groupData['price'].min()
        frame = {'volume': volsum, 'vwap': vwap, 'open': o, 'close': c, 'high': high, 'low': low}
        frame = pd.DataFrame(frame)
        return frame

In [44]:
try:
  try:
    taq = TAQ('hourAll.csv')
  except:
    import io
    taq = TAQ(io.BytesIO(uploaded['QQQ1Day.csv']))
except:
    taq = TAQ('QQQ1Day.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [45]:
rawData = taq.rawData
rawData.head(100)

,DATE,TIME_M,EX,SYM_ROOT,SYM_SUFFIX,TR_SCOND,SIZE,PRICE,TR_CORR,TR_SEQNUM,TR_ID,TR_SOURCE,TR_RF
0,20200102,9:29:57.929553117,P,A,NaN,T,100,85.8000,0,304701,52983525035336,C,NaN
1,20200102,9:29:57.930146264,T,A,NaN,T,100,85.8000,0,304801,3472338191833380689,C,NaN
2,20200102,9:30:00.436516480,N,A,NaN,O,24994,85.9000,0,334001,52983525028169,C,NaN
3,20200102,9:30:00.436537082,N,A,NaN,Q,24994,85.9000,0,334101,52983525028170,C,NaN
4,20200102,9:30:00.438161828,N,A,NaN,NaN,100,85.9000,0,334801,52983525028175,C,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20200102,9:30:34.350369795,D,A,NaN,NaN,100,85.8000,0,560301,71675224217399,C,T
96,20200102,9:30:34.350929223,D,A,NaN,NaN,100,85.8275,0,560401,71675224217400,C,T
97,20200102,9:30:34.366052905,D,A,NaN,NaN,100,85.8600,0,560501,71675224217415,C,T
98,20200102,9:30:34.402349585,D,A,NaN,NaN,100,85.8000,0,560601,71675224217421,C,T


Exchange CODE Description

A NYSE MKT LLC

B NASDAQ OMX BX, Inc.

C National Stock Exchange Inc. (NSX)

D Financial Industry Regulatory Authority, Inc.(FINRA ADF)

I International Securities Exchange, LLC (ISE)

J Bats EDGA Exchange, INC

K Bats EDGX Exchange, Inc.

M Chicago Stock Exchange, Inc. (CHX)

N New York Stock Exchange LLC

P NYSE Arca, Inc.

S Consolidated Tape System

T NASDAQ Stock Exchange, LLC (in Tape A, B securities)

Q NASDAQ Stock Exchange, LLC (in Tape C securities)

V The Investors’ Exchange, LLC (IEX)

W Chicago Broad Options Exchange, Inc.(CBOE)

X NASDAQ OMX PSX, Inc. LLC

Y Bats BYX Exchange, Inc.

Z Bats BZX Exchange, Inc.

In [47]:
taq.data.head(30)

,ticker,exchange,volume,price,priceXvolume,timestamp,date,ticks
0,A,P,100,85.80,8580.00,2020-01-02 09:29:00,2020-01-02,1.0
1,A,T,100,85.80,8580.00,2020-01-02 09:29:00,2020-01-02,1.0
2,A,N,24994,85.90,2146984.60,2020-01-02 09:30:00,2020-01-02,1.0
3,A,N,24994,85.90,2146984.60,2020-01-02 09:30:00,2020-01-02,1.0
4,A,N,100,85.90,8590.00,2020-01-02 09:30:00,2020-01-02,1.0
5,A,N,2,85.55,171.10,2020-01-02 09:30:00,2020-01-02,-1.0
6,A,N,157,85.52,13426.64,2020-01-02 09:30:00,2020-01-02,-1.0
7,A,A,100,85.56,8556.00,2020-01-02 09:30:00,2020-01-02,1.0
8,A,N,41,85.51,3505.91,2020-01-02 09:30:00,2020-01-02,-1.0
9,A,A,100,85.20,8520.00,2020-01-02 09:30:00,2020-01-02,-1.0


In [28]:
taq.timeBars

volume       vwap  ...     high      low
ticker timestamp                               ...                  
A      2020-01-02 09:29:00     200  85.800000  ...  85.8000  85.8000
       2020-01-02 09:30:00   83057  85.884028  ...  85.9500  85.2000
       2020-01-02 09:31:00   38976  85.965001  ...  86.0400  85.7800
       2020-01-02 09:32:00    7126  86.056736  ...  86.2000  85.9700
       2020-01-02 09:33:00    9293  86.140407  ...  86.2500  85.9000
...                            ...        ...  ...      ...      ...
ZYXI   2020-01-02 10:25:00     105   7.950476  ...   7.9600   7.9500
       2020-01-02 10:26:00     300   7.947133  ...   7.9600   7.9390
       2020-01-02 10:27:00     100   7.926500  ...   7.9265   7.9265
       2020-01-02 10:28:00     300   7.917633  ...   7.9229   7.9100
       2020-01-02 10:29:00     100   7.940000  ...   7.9400   7.9400

[267711 rows x 6 columns]

In [42]:
(taq.data['price'].diff()/abs(taq.data['price'].diff())).fillna(method = 'ffill')

0          NaN
1          NaN
2          1.0
3          1.0
4          1.0
          ... 
7220599   -1.0
7220600   -1.0
7220601   -1.0
7220602   -1.0
7220603    1.0
Name: price, Length: 7220604, dtype: float64

In [35]:
taq.data['ticker'].shift(1).ne(taq.data['ticker'])


0           True
1          False
2          False
3          False
4          False
           ...  
7220599    False
7220600    False
7220601    False
7220602    False
7220603    False
Name: ticker, Length: 7220604, dtype: bool

## Ticker Specific

In [0]:
QQQ = data[data.SYM_ROOT == 'QQQ']

In [0]:
QQQ.shape

(35788, 13)

In [0]:
QQQ.EX.value_counts()

Q    10001
D     6265
P     6156
Z     4299
K     3659
Y     1193
N     1133
X      879
B      698
J      653
V      601
C      164
A       70
M       17
Name: EX, dtype: int64

In [0]:
IBM = data[data.SYM_ROOT == 'IBM']

In [0]:
IBM.EX.value_counts()

D    2129
Z    1283
N    1101
T    1032
K     686
P     582
Y     395
J     327
V     116
B      38
C      18
X      11
A       2
M       1
Name: EX, dtype: int64

In [0]:
IBM[IBM.DATE == 20200103]

,DATE,TIME_M,EX,SYM_ROOT,SYM_SUFFIX,TR_SCOND,SIZE,PRICE,TR_CORR,TR_SEQNUM,TR_ID,TR_SOURCE,TR_RF
